# Importing libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = 'cache/models--meta-llama--CodeLlama-7b-Instruct-hf/snapshots/4ce0c40b2ea823bd1d8f1f3fd5bc8a7e80d749bc'

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Wed Dec 11 15:19:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               80W / 484W|  10449MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the NS3 code for the smart_agriculture scenario.

In [3]:
with open("projects/testbed-at-JNCA/blueprint/sample_blueprint.json") as f:
        sample_blueprint = f.read()
with open("projects/testbed-at-JNCA/blueprint/sample_ns3_code.cc") as f:
        sample_ns3_code = f.read()
with open("projects/testbed-at-JNCA/blueprint/smart_agriculture.json") as f:
        test_blueprint = f.read()

In [4]:
prompt = f"""
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-sink-helper.h"
            #include "ns3/ssid.h"
            #include "ns3/internet-stack-helper.h"
            #include "ns3/ipv4-address-helper.h"

        Set up mobility for WiFi nodes in this way:

        Ptr<ListPositionAllocator> allocatorAPWiFi = CreateObject<ListPositionAllocator>();
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorAPWiFi);

        Ptr<ListPositionAllocator> allocatorStaWiFi = CreateObject<ListPositionAllocator>();
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorStaWiFi);

        mobility.Install(wifiNodes);

        If <height> not given set 1.5 meters

    Generate only the code without "```cpp" quote it will be saved to a .cc file:

            SAMPLE BLUEPRINT:

            {sample_blueprint}

            SAMPLE NS3 CODE:

            {sample_ns3_code}

            TEST BLUEPRINT:

            {test_blueprint}

            NS3 CODE:

            """
print(prompt)


    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-s

In [5]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:2134: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> 
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/pack

## Save the .cc code into a file.

In [6]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/domain-expert-prompting/codellama/smart_agriculture-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_city scenario.

In [7]:
with open("projects/testbed-at-JNCA/blueprint/smart_city.json") as f:
        test_blueprint = f.read()

In [8]:
prompt = f"""
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-sink-helper.h"
            #include "ns3/ssid.h"
            #include "ns3/internet-stack-helper.h"
            #include "ns3/ipv4-address-helper.h"

        Set up mobility for WiFi nodes in this way:

        Ptr<ListPositionAllocator> allocatorAPWiFi = CreateObject<ListPositionAllocator>();
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorAPWiFi);

        Ptr<ListPositionAllocator> allocatorStaWiFi = CreateObject<ListPositionAllocator>();
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorStaWiFi);

        mobility.Install(wifiNodes);

        If <height> not given set 1.5 meters

    Generate only the code without "```cpp" quote it will be saved to a .cc file:

            SAMPLE BLUEPRINT:

            {sample_blueprint}

            SAMPLE NS3 CODE:

            {sample_ns3_code}

            TEST BLUEPRINT:

            {test_blueprint}

            NS3 CODE:

            """
print(prompt)


    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-s

In [9]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/pack

## Save the .cc code into a file.

In [10]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/domain-expert-prompting/codellama/smart_city-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_home scenario.

In [11]:
with open("projects/testbed-at-JNCA/blueprint/smart_home.json") as f:
        test_blueprint = f.read()

In [12]:
prompt = f"""
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-sink-helper.h"
            #include "ns3/ssid.h"
            #include "ns3/internet-stack-helper.h"
            #include "ns3/ipv4-address-helper.h"

        Set up mobility for WiFi nodes in this way:

        Ptr<ListPositionAllocator> allocatorAPWiFi = CreateObject<ListPositionAllocator>();
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorAPWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorAPWiFi);

        Ptr<ListPositionAllocator> allocatorStaWiFi = CreateObject<ListPositionAllocator>();
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        allocatorStaWiFi->Add(Vector(<lat>, <lon>, <height>)); // position depends on blueprint elements
        mobility.SetPositionAllocator(allocatorStaWiFi);

        mobility.Install(wifiNodes);

        If <height> not given set 1.5 meters

    Generate only the code without "```cpp" quote it will be saved to a .cc file:

            SAMPLE BLUEPRINT:

            {sample_blueprint}

            SAMPLE NS3 CODE:

            {sample_ns3_code}

            TEST BLUEPRINT:

            {test_blueprint}

            NS3 CODE:

            """
print(prompt)


    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/packet-s

In [13]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS-3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT.

    Do Not use WiFi is not used in the TEST BLUEPRINT
    Do Not use LoRa is not used in the TEST BLUEPRINT

    In case of WiFi nodes take into accounts this:
    - wifi.SetStandard(WIFI_STANDARD_80211a); // Set Wi-Fi standard to 802.11a
    - declare YansWifiChannelHelper with "wifichannel", to avoid conflict in the code variables
    - initialize YansWifiChannelHelper using "YansWifiChannelHelper::Default ()"
    - "YansWifiPhyHelper phy = YansWifiPhyHelper::Default();" is wrong use "YansWifiPhyHelper phy;" instead
    - add a tracker for Udp and LoRa to save experiment outputs to file.
    FOR SCENARIOS WITH WIFI NODES:
        must includes following libraries:
            #include "ns3/yans-wifi-helper.h"
            #include "ns3/on-off-helper.h"
            #include "ns3/inet-socket-address.h"
            #include "ns3/pack

## Save the .cc code into a file.

In [14]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/domain-expert-prompting/codellama/smart_home-1_shot.cc", "w") as file:
    file.write(s)